In [181]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

In [323]:
from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
import geopandas as gpd

graph = Graph.from_file("PA/full/PA_VTD.shp")




In [183]:
elections = [
    Election("SEN10", {"Democratic": "SEN10D", "Republican": "SEN10R"}),
    Election("SEN12", {"Democratic": "USS12D", "Republican": "USS12R"}),
    Election("SEN16", {"Democratic": "T16SEND", "Republican": "T16SENR"}),
    Election("PRES12", {"Democratic": "PRES12D", "Republican": "PRES12R"}),
    Election("PRES16", {"Democratic": "T16PRESD", "Republican": "T16PRESR", "Other": "T16PRESOTH"})
]

In [225]:

# Population updater, for computing how close to equality the district
# populations are. "TOT_POP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TOT_POP", alias="population"), 
                "whites": updaters.Tally("WHITE_POP", alias="whites"),
                "dem": updaters.Tally("T16PRESD", alias="dem"),
                "rep": updaters.Tally("T16PRESR", alias="rep"),
                "oth": updaters.Tally("T16PRESOTH", alias="oth"), 
                "d12": updaters.Tally("PRES12D", alias = "d12"), 
                "r12": updaters.Tally("PRES12R", alias = "r12"), 
                "o12": updaters.Tally("PRES12O", alias = "o12")}

# Election updaters, for computing election results using the vote totals
# from our shapefile.
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

In [329]:
initial_partition = GeographicPartition(
    graph,
    assignment="REMEDIAL_P",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOT_POP", alias="population"),
        "whites": Tally("WHITE_POP", alias="whites"),
        "dem": Tally("T16PRESD", alias="dem"),
        "rep": Tally("T16PRESR", alias="rep"),
        "oth": Tally("T16PRESOTH", alias="oth"),
        "PRES16": election
    }
)


In [330]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.03)

In [332]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        pop_constraint,
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=1002
)

In [331]:
# The ReCom proposal needs to know the ideal population for the districts so that
# we can improve speed by bailing early on unbalanced partitions.

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

In [336]:
j = 1 

aaa = """for partition in chain: 
    
    if j%10 == 0: 
        shp = "./IA/" + str(int(j/10)) + "/districts.shp"
        ft = "./IA/" + str(int(j/10)) + "/features.csv"
        assignment = {}
        for i in partition.assignment: 
            assignment[i] = partition.assignment[i]
        df2 = df.copy(deep = True)
        print("Copied", j)
        df2["CD"] = df2.index.map(assignment)
        print("Mapped", j)
        gf = df2.dissolve(by='CD')
        gf["CD"] = gf.index
        print("Dissolved", j)
        gf.to_file(shp) 
        d = {}
        print("Features", j)
        for i in range(1, 5): 
            d[i] = {}
            d[i]["Population"] = partition["population"][i]
            d[i]["White"] = partition["whites"][i]
            d[i]["Democratic"] = partition["dem"][i]
            d[i]["Republican"] = partition["rep"][i]
            d[i]["Other"] = partition["oth"][i]

        dft = pd.DataFrame(d).T
        dft.to_csv(ft)
        print("feat!!!")
    print(j)
    j += 1
"""


In [147]:
assignment = {}
for i in initial_partition.assignment: 
    assignment[i] = initial_partition.assignment[i]

In [325]:
election = Election("PRES16", {"Dem": "PRES16D", "Rep": "PRES16R"})

initial_partition = Partition(
    graph,
    assignment="REMEDIAL_P",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOT_POP", alias="population"),
        "whites": Tally("WHITE_POP", alias="whites"),
        "dem": Tally("T16PRESD", alias="dem"),
        "rep": Tally("T16PRESR", alias="rep"),
        "oth": Tally("T16PRESOTH", alias="oth"),
        "PRES16": election
    }
)

In [327]:
d = {}

for i in range(1, 19): 
    d[i] = {}
    d[i]["Population"] = initial_partition["population"][i]
    d[i]["White"] = initial_partition["whites"][i]
    d[i]["Democratic"] = initial_partition["dem"][i]
    d[i]["Republican"] = initial_partition["rep"][i]
    d[i]["Other"] = initial_partition["oth"][i]

    
dft = pd.DataFrame(d).T
dft.to_csv("PA/d18/features.csv")

In [337]:
d = {}

for i in range(1, 19): 
    k = str(i)
    d[i] = {}
    d[i]["Population"] = initial_partition["population"][k]
    d[i]["White"] = initial_partition["whites"][k]
    d[i]["Democratic"] = initial_partition["dem"][k]
    d[i]["Republican"] = initial_partition["rep"][k]
    d[i]["Other"] = initial_partition["oth"][k]

In [90]:
d

{}

In [52]:
districts = {}
for i in range(1, 19): 
    districts[i] = []
    for k in initial_partition.parts[i]:
        districts[i].append(k)

In [54]:
with open("assignment.json", 'w') as outfile:
    json.dump(districts, outfile)

In [49]:
import json

In [55]:
graph

<Graph [8921 nodes, 25228 edges]>

In [78]:
len(df)

8921

In [127]:
initial_partition.assignment[0]

14